### Imports

In [46]:
# import reddit api wrapper
import praw
import pandas as pd
import datetime

from textblob import TextBlob
import altair as alt

# note that the %load_ext autoreload line only needs to be be run once
%load_ext autoreload

In [113]:
# by including this autoreload command, we only need to re-import Spatial_Joins if we make/save changes to the original py file
%autoreload
from nfl_gamethreads import nfl_gamethreads

### Set Up Reddit Instance

In [2]:
# access password and client secret id via local files
with open('pw.txt', 'r') as file1:
    pw = file1.read()

with open('client_secret.txt', 'r') as file2:
    cs = file2.read()

# create a praw Reddit instance with app credentials and secret info passed through
reddit = praw.Reddit(
    client_id="XbesrQBvKymjgLdgg_D6lA",
    client_secret=cs,
    user_agent="NFLTextAnalysis/0.0.1",
    username="ta_api",
    password=pw
)

# store an instance of the Redditor object for the gamethread account
nfl_bot = reddit.redditor("nfl_gamethread")

### Get Raw Data

In [3]:
# initialize a new list for storing gamethread data
gamethread_list = []

# for each gamethread, store the id, name, and date
for submission in nfl_bot.submissions.new(limit=150):
    gamethread_list.append((str(submission.id), str(submission.title), submission.created_utc))

gamethread_df = pd.DataFrame(gamethread_list, columns=['id','title','date'])

### Pre-process Data

In [5]:
# transform the date column from UTC timestamp to only the date
gamethread_df['date'] = gamethread_df['date'].apply(lambda utc_entry: datetime.datetime.utcfromtimestamp(utc_entry))
gamethread_df['date'] = gamethread_df['date'].dt.date

In [24]:
# remove pre and post game threads, the superbowl halftime discussion, and the pro bowl discussion
gamethread_df = gamethread_df[gamethread_df["title"].str.contains("Pre|Post|Halftime|Pro Bowl|Super Bowl|RedZone")==False].copy()

### Filter By Week

In [97]:
print(gamethread_df.shape)
gamethread_df_17 = nfl_gamethreads.filter_by_week(gamethread_df=gamethread_df, week=17)
print(gamethread_df_17.shape)

(64, 3)
(14, 3)


### Get Gameday Data

In [98]:
gamethread_df_17.reset_index(inplace=True, drop=True)

In [99]:
gamedata = [nfl_gamethreads.get_game_data(reddit, each_id) for each_id in gamethread_df_17['id']]

gamedata_df = pd.DataFrame(gamedata, columns=['submission_id', 'title', 'home_team_full', 'home_team_wins', 'home_team_losses', 'home_team_ties', 
                       'away_team_full', 'away_team_wins', 'away_team_losses', 'away_team_ties',
                       'home_score', 'away_score', 'winner', 'combined_score', 'diff', 'pred_winner', 'pred_diff', 'pred_ou'])
gamedata_df.sample(10)

,submission_id,title,home_team_full,home_team_wins,home_team_losses,home_team_ties,away_team_full,away_team_wins,away_team_losses,away_team_ties,home_score,away_score,winner,combined_score,diff,pred_winner,pred_diff,pred_ou
13,ruejb4,Los Angeles Rams (11-4) at Baltimore Ravens (8-7),Baltimore Ravens,8,7,0,Los Angeles Rams,11,4,0,19,20,Rams,39,1,Baltimore,7.0,46.5
1,ruj6sd,Detroit Lions (2-12-1) at Seattle Seahawks (5-10),Seattle Seahawks,5,10,0,Detroit Lions,2,12,1,51,29,Seahawks,80,22,Seattle,9.0,41.0
0,ruj6th,Arizona Cardinals (10-5) at Dallas Cowboys (11-4),Dallas Cowboys,11,4,0,Arizona Cardinals,10,5,0,22,25,Cardinals,47,3,Dallas,6.5,52.5
4,ruipqi,Houston Texans (4-11) at San Francisco 49ers (...,San Francisco 49ers,8,7,0,Houston Texans,4,11,0,23,7,49ers,30,16,San Francisco,13.5,44.0
5,ruejqh,Philadelphia Eagles (8-7) at Washington Footba...,Washington Football Team,6,9,0,Philadelphia Eagles,8,7,0,16,20,Eagles,36,4,Washington,6.0,44.5
9,ruejhw,Las Vegas Raiders (8-7) at Indianapolis Colts ...,Indianapolis Colts,9,6,0,Las Vegas Raiders,8,7,0,20,23,Raiders,43,3,Indianapolis,8.5,46.5
8,ruejkt,Kansas City Chiefs (11-4) at Cincinnati Bengal...,Cincinnati Bengals,9,6,0,Kansas City Chiefs,11,4,0,34,31,Bengals,65,3,Cincinnati,3.5,51.0
11,ruejf3,Miami Dolphins (8-7) at Tennessee Titans (10-5),Tennessee Titans,10,5,0,Miami Dolphins,8,7,0,34,3,Titans,37,31,Tennessee,3.0,41.0
7,ruejna,New York Giants (4-11) at Chicago Bears (5-10),Chicago Bears,5,10,0,New York Giants,4,11,0,29,3,Bears,32,26,Chicago,6.5,36.5
2,ruj6pr,Carolina Panthers (5-10) at New Orleans Saints...,New Orleans Saints,7,8,0,Carolina Panthers,5,10,0,18,10,Saints,28,8,New Orleans,7.0,37.0
